In [18]:
!pip install faker

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [19]:
from faker import Faker
faker = Faker()

In [20]:
positive_custom_list = [
    'this is good', 
    'i like it', 
    'very delicious', 
    'i would recommend this to my friends',
    'food in the restaurant',
    'spaghetti chicken soup',
    'dinner time',
    'tastes good',
    'donut',
    'very good',
    'impressive']

In [21]:
def generate_positive_sentences():
    return faker.sentence(
        ext_word_list=positive_custom_list
    )

In [22]:
negative_custom_list = [
    'this is bad', 
    'i hate it', 
    'there are better restaurants out there', 
    'i will not recommend this to my friends',
    'food in the restaurant',
    'spaghetti chicken soup',
    'dinner time',
    'tastes bad',
    'donut',
    'very bad',
    'not impressive']

In [23]:
def generate_negative_sentences():
    return faker.sentence(
        ext_word_list=negative_custom_list
    )

In [24]:
positive_sentences = []

for i in range(0, 1000):
    item = generate_positive_sentences()
    item = item.replace(".","")
    positive_sentences.append(item)

In [33]:
positive_sentences

['Dinner Time this is good this is good this is good i like it very delicious food in the restaurant',
 'I Like It food in the restaurant donut',
 'Tastes Good i like it tastes good food in the restaurant i would recommend this to my friends',
 'Very Delicious very good very good spaghetti chicken soup i would recommend this to my friends i like it',
 'Donut i would recommend this to my friends this is good impressive spaghetti chicken soup very delicious impressive',
 'Very Delicious dinner time i like it very good i would recommend this to my friends food in the restaurant food in the restaurant',
 'Donut spaghetti chicken soup impressive donut',
 'I Like It food in the restaurant dinner time very good',
 'Food In The Restaurant i like it i would recommend this to my friends very delicious this is good tastes good i like it',
 'Spaghetti Chicken Soup i like it i like it food in the restaurant',
 'Very Good tastes good this is good very good',
 'I Would Recommend This To My Friends i 

In [25]:
negative_sentences = []

for i in range(0, 1000):
    item = generate_negative_sentences()
    item = item.replace(".","")
    negative_sentences.append(item)

In [34]:
negative_sentences

['Very Bad food in the restaurant food in the restaurant very bad this is bad spaghetti chicken soup',
 'Spaghetti Chicken Soup i hate it this is bad this is bad tastes bad spaghetti chicken soup food in the restaurant spaghetti chicken soup',
 'Donut food in the restaurant spaghetti chicken soup',
 'Dinner Time not impressive very bad i will not recommend this to my friends dinner time donut very bad',
 'This Is Bad very bad very bad not impressive',
 'This Is Bad dinner time not impressive there are better restaurants out there this is bad i hate it food in the restaurant',
 'Very Bad donut tastes bad there are better restaurants out there there are better restaurants out there food in the restaurant food in the restaurant',
 'There Are Better Restaurants Out There tastes bad very bad i will not recommend this to my friends i will not recommend this to my friends',
 'Spaghetti Chicken Soup spaghetti chicken soup dinner time there are better restaurants out there dinner time there are

In [26]:
import pandas as pd 

positive_df = pd.DataFrame(
    positive_sentences, 
    columns=['text']
)

positive_df.insert(
    0, 
    "label", 
    "__label__positive"
)

In [35]:
positive_df

,label,text
0,__label__positive,Dinner Time this is good this is good this is ...
1,__label__positive,I Like It food in the restaurant donut
2,__label__positive,Tastes Good i like it tastes good food in the ...
3,__label__positive,Very Delicious very good very good spaghetti c...
4,__label__positive,Donut i would recommend this to my friends thi...
...,...,...
995,__label__positive,Dinner Time dinner time dinner time donut i li...
996,__label__positive,This Is Good this is good this is good impress...
997,__label__positive,Dinner Time very good spaghetti chicken soup t...
998,__label__positive,Food In The Restaurant dinner time donut spagh...


In [27]:
negative_df = pd.DataFrame(
    negative_sentences, 
    columns=['text']
)
negative_df.insert(
    0, 
    "label", 
    "__label__negative"
)

In [36]:
negative_df

,label,text
0,__label__negative,Very Bad food in the restaurant food in the re...
1,__label__negative,Spaghetti Chicken Soup i hate it this is bad t...
2,__label__negative,Donut food in the restaurant spaghetti chicken...
3,__label__negative,Dinner Time not impressive very bad i will not...
4,__label__negative,This Is Bad very bad very bad not impressive
...,...,...
995,__label__negative,Very Bad not impressive dinner time donut tast...
996,__label__negative,I Will Not Recommend This To My Friends not im...
997,__label__negative,Spaghetti Chicken Soup i will not recommend th...
998,__label__negative,Very Bad dinner time this is bad this is bad d...


In [28]:
all_df = pd.concat(
    [positive_df, negative_df], 
    ignore_index=True
)

In [29]:
from sklearn.model_selection import train_test_split
train_val_df, test_df = train_test_split(
    all_df, 
    test_size=0.2
) 
train_df, val_df = train_test_split(
    train_val_df, 
    test_size=0.25
)

In [30]:
!mkdir tmp 
train_df.to_csv(
    "tmp/synthetic.train.txt", 
    header=False, 
    index=False, 
    sep=" ", 
    quotechar=" "
)
val_df.to_csv(
    "tmp/synthetic.validation.txt", 
    header=False, 
    index=False, 
    sep=" ", 
    quotechar=" "
) 
test_df.to_csv(
    "tmp/synthetic.test.txt", 
    header=False, 
    index=False, 
    sep=" ", 
    quotechar=" "
)

mkdir: cannot create directory ‘tmp’: File exists


In [37]:
!head tmp/synthetic.train.txt

__label__positive  Food  In  The  Restaurant  donut  donut  this  is  good  very  delicious  very  delicious  food  in  the  restaurant  dinner  time 
__label__positive  I  Like  It  very  good  very  good  spaghetti  chicken  soup  impressive  spaghetti  chicken  soup 
__label__positive  Donut  food  in  the  restaurant  donut  very  delicious  spaghetti  chicken  soup 
__label__positive  Very  Delicious  impressive  dinner  time  i  like  it  dinner  time 
__label__negative  There  Are  Better  Restaurants  Out  There  dinner  time  food  in  the  restaurant  i  will  not  recommend  this  to  my  friends  there  are  better  restaurants  out  there  this  is  bad  donut 
__label__negative  There  Are  Better  Restaurants  Out  There  this  is  bad  donut  donut  spaghetti  chicken  soup 
__label__positive  Very  Good  i  like  it  i  like  it 
__label__positive  Very  Delicious  impressive  impressive  i  like  it  very  good  impressive 
__label__positive  Spaghetti  Chicken  Soup 

In [31]:
s3_bucket = "sagemaker-cookbook-bucket"
prefix = "chapter08"
!aws s3 cp tmp/synthetic.train.txt s3://{s3_bucket}/{prefix}/input/synthetic.train.txt 
!aws s3 cp tmp/synthetic.validation.txt s3://{s3_bucket}/{prefix}/input/synthetic.validation.txt

upload: tmp/synthetic.train.txt to s3://sagemaker-cookbook-bucket/chapter08/input/synthetic.train.txt
upload: tmp/synthetic.validation.txt to s3://sagemaker-cookbook-bucket/chapter08/input/synthetic.validation.txt


In [32]:
%store test_df
%store s3_bucket
%store prefix

Stored 'test_df' (DataFrame)
Stored 's3_bucket' (str)
Stored 'prefix' (str)
